In [1]:
# mount drive to gain access to data and modules, run once
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/colab_scripts')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import data_preprocessing_v2 as dpp
import imp

imp.reload(dpp)

<module 'data_preprocessing_v2' from '/content/drive/MyDrive/colab_scripts/data_preprocessing_v2.py'>

In [3]:
train, valid, test = dpp.clean_freehand_datasets()

In [4]:
# now extract features and targets and scale data (and also downsample by factor of 5)
train_x, train_y = dpp.get_scaled_features_n_targets(train, downsample = True)
valid_x, valid_y = dpp.get_scaled_features_n_targets(valid, downsample = True)
test_x, test_y = dpp.get_scaled_features_n_targets(test, downsample = True)
# Reshape for stateful learning
train_x = dpp.reshape(train_x)
train_y = dpp.reshape(train_y)
valid_x = dpp.reshape(valid_x)
valid_y = dpp.reshape(valid_y)
test_x = dpp.reshape(test_x)
test_y = dpp.reshape(test_y)

# extract pressure only for targets
for i in range(len(train_x)):
  train_x[i] = train_x[i][:,:,-1:]
  valid_x[i] = valid_x[i][:,:,-1:]
  test_x[i] = test_x[i][:,:,-1:]

In [5]:
# now Implement the model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
import tensorflow as tf
from keras.regularizers import l2

window_size = train_x[0].shape[1]
num_features = train_x[0].shape[2]

# attach GPU and implement model
with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(LSTM(256,
                 return_sequences=True,
                 activation='tanh',
                 input_shape = (window_size,num_features),
                 batch_size = 1,
                 stateful=True))
  model.add(LSTM(256,
                 return_sequences=True,
                 activation='tanh',
                 input_shape = (window_size,num_features),
                 batch_size = 1,
                 stateful=True))
  # model.add(LSTM(256, return_sequences=True, activation='tanh', batch_size = 1, stateful = True))
  model.add(Dense(1))
  # my_optimizer = Adam(learning_rate = 0.001)
  model.compile(optimizer='adam', loss='mse')
  # print model summary
  model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, 1, 256)               264192    
                                                                 
 lstm_1 (LSTM)               (1, 1, 256)               525312    
                                                                 
 dense (Dense)               (1, 1, 1)                 257       
                                                                 
Total params: 789761 (3.01 MB)
Trainable params: 789761 (3.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
import tensorflow as tf
model = tf.keras.models.load_model("drive/MyDrive/colab_data/trained_models/L2_N256_N256_DFreehand_univariate/L2_N256_N256_DFreehand_univariate_e225.keras")

In [7]:
epochs_done = 226
epochs = 500
train_data_count = len(train_x)
model_path = 'drive/MyDrive/colab_data/trained_models/L2_N256_N256_DFreehand_univariate/'
model_name = 'L2_N256_N256_DFreehand_univariate_e'
dataset_names = ["freehand tt","freehand static 3", "freehand static 9", "freehand sin", "static_0r0p tt"]

In [ ]:
for i in range(epochs_done, epochs):
  print("Training Epoch: " + str(i))
  for j in range(train_data_count):
    print("Fitting to dataset - " + dataset_names[j])

    model.fit(train_x[j],train_y[j],epochs=1, batch_size = 1, validation_data=(valid_x[j], valid_y[j]), verbose=1, shuffle=False)
    model.reset_states()

  if i % 5 == 0:
    model.save(model_path + model_name + str(i) + '.keras')

model.save(model_path + model_name + 'FINAL.keras')